In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['drug_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['drug_two_year'].values

In [3]:
#### CART
depth = [8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.7]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [5]:
cart_summary['FL_score'], ebm_summary['FL_score']

(0.6466788008830898, 0.6217845360057549)

### Lasso Stumps

In [6]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['drug_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['drug_two_year'].values

## columns 
cols = KY_X.columns

In [8]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.004, cols, 816)
single_stump_model['features']

['sex1',
 'age_at_current_charge20.0',
 'age_at_current_charge21.0',
 'age_at_current_charge28.0',
 'age_at_current_charge32.0',
 'age_at_current_charge34.0',
 'age_at_current_charge35.0',
 'age_at_current_charge39.0',
 'age_at_current_charge45.0',
 'age_at_current_charge46.0',
 'age_at_current_charge47.0',
 'age_at_current_charge48.0',
 'age_at_current_charge70.0',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_arrest6',
 'p_arrest7',
 'p_charges1',
 'p_charges3',
 'p_charges4',
 'p_charges5',
 'p_charges6',
 'p_violence1',
 'p_felony1',
 'p_felony2',
 'p_felony3',
 'p_misdeassult1',
 'p_traffic1',
 'p_drug1',
 'p_drug2',
 'p_drug3',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1',
 'six_month1',
 'one_year1',
 'five_year1']

In [9]:
c_grid={'C': [0.001, 0.002, 0.003, 0.004]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [10]:
stumps_summary['best_params'], stumps_summary['FL_score'], stumps_summary['auc_diffs']

({'C': 0.004}, 0.6563329447076626, 0.0006186501086570262)

### RiskSLIM

In [11]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['drug_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['drug_two_year'].values

## columns 
cols = KY_X.columns

In [25]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 
                                        0.0008, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

15

In [22]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [23]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'drug_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000, 
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 01:53 PM | 243 rows in lookup table
11/12/19 @ 01:53 PM | ------------------------------------------------------------
11/12/19 @ 01:53 PM | runnning initialization procedure
11/12/19 @ 01:53 PM | ------------------------------------------------------------
11/12/19 @ 01:53 PM | CPA produced 2 cuts
11/12/19 @ 01:53 PM | running naive rounding on 6 solutions
11/12/19 @ 01:53 PM | best objective value: 0.6931
11/12/19 @ 01:53 PM | rounding produced 5 integer solutions
11/12/19 @ 01:53 PM | best objective value is 0.6931
11/12/19 @ 01:53 PM | running sequential rounding on 6 solutions
11/12/19 @ 01:53 PM | best objective value: 0.6931
11/12/19 @ 01:53 PM | sequential rounding produced 0 integer solutions
11/12/19 @ 01:53 PM | polishing 5 solutions
11/12/19 @ 01:53 PM | best objective value: 0.6931
11/12/19 @ 01:53 PM | polishing produced 5 integer solutions
11/12/19 @ 01:53 PM | initialization produced 9 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 01:53 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3615.
Tried aggregator 1 time.
Reduced MIP has 40 rows, 42 columns, and 117 nonzeros.
Reduced MIP has 19 binaries, 21 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 610052 200739        0.3549     7        0.3579        0.3488  2871627    2.55%           rho_2 D 610052 610051     32
 624182 203364        0.3553     9        0.3579        0.3489  2931346    2.53%          rho_16 D 624182 624180     40
 638280 206998        0.3521     1        0.3579        0.3489  2995435    2.52%          rho_12 U 638280 638278     40
 653877 211115        0.3490     7        0.3579        0.3490  3057469    2.51%           rho_0 D 653877 653864     30
 669042 213595        0.3554    11        0.3579        0.3490  3120262    2.49%           rho_5 U 669042 669041     29
 684132 217308        0.3549     8        0.3579        0.3491  3184702    2.47%           rho_4 D 684132 684131     39
Elapsed time = 175.16 sec. (168107.77 ticks, tree = 62.03 MB, solutions = 5)
 698405 220242        cutoff              0.3579        0.3491  3253339    2.46%          rho_16 D 698405 698404     43
 712750 222686        0.3550     3        0.3579        0.3492  3324143    2.44%   

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 02:02 PM | 243 rows in lookup table
11/12/19 @ 02:02 PM | ------------------------------------------------------------
11/12/19 @ 02:02 PM | runnning initialization procedure
11/12/19 @ 02:02 PM | ------------------------------------------------------------
11/12/19 @ 02:02 PM | CPA produced 2 cuts
11/12/19 @ 02:02 PM | running naive rounding on 5 solutions
11/12/19 @ 02:02 PM | best objective value: 0.6931
11/12/19 @ 02:02 PM | rounding produced 5 integer solutions
11/12/19 @ 02:02 PM | best objective value is 0.6931
11/12/19 @ 02:02 PM | running sequential rounding on 5 solutions
11/12/19 @ 02:02 PM | best objective value: 0.6931
11/12/19 @ 02:02 PM | sequential rounding produced 0 integer solutions
11/12/19 @ 02:02 PM | polishing 5 solutions
11/12/19 @ 02:02 PM | best objective value: 0.6931
11/12/19 @ 02:02 PM | polishing produced 5 integer solutions
11/12/19 @ 02:02 PM | initialization produced 9 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:02 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3633.
Tried aggregator 1 time.
Reduced MIP has 40 rows, 42 columns, and 117 nonzeros.
Reduced MIP has 19 binaries, 21 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

...

 582842 231673        cutoff              0.3588        0.3471  3571158    3.26%           rho_6 D 582842 582841     28
 599555 237305        0.3524    17        0.3588        0.3471  3640126    3.24%           rho_4 U 599555 599554     31
 616340 243834        0.3567     9        0.3588        0.3472  3716515    3.23%          rho_10 D 616340 616339     22
 631828 247382        0.3474    15        0.3588        0.3473  3786464    3.20%           rho_6 U 631828 466832     22
 647769 251310        cutoff              0.3588        0.3474  3855592    3.17%           rho_0 U 647769 647767     17
 663969 256616        0.3493    16        0.3588        0.3475  3928726    3.15%           rho_0 U 663969 115094     19
Elapsed time = 178.59 sec. (168117.08 ticks, tree = 72.43 MB, solutions = 6)
 678962 260499        0.3524     1        0.3588        0.3476  4001150    3.12%          rho_10 U 678962 464501     38
 694559 264645        0.3582    15        0.3588        0.3477  4073193    3.09%   

 2307403 282247        0.3550    15        0.3578        0.3546 11394871    0.89%          rho_14 U 2307403 899476     26
Elapsed time = 615.39 sec. (625903.00 ticks, tree = 90.58 MB, solutions = 7)
 2320039 276836        0.3578    13        0.3578        0.3546 11442080    0.87%           rho_8 U 2320039 153663     32
 2332420 271244        0.3555    16        0.3578        0.3547 11487705    0.85%           rho_6 U 2332420 2126994     37
 2344418 264809        cutoff              0.3578        0.3548 11532307    0.83%        alpha_16 U 2344418 1502286     28
 2356280 258706        0.3568     4        0.3578        0.3548 11576789    0.81%           rho_4 U 2356280 161676     13
 2368864 253570        cutoff              0.3578        0.3549 11620443    0.79%           rho_8 D 2368864 1653227     38
 2381660 248400        cutoff              0.3578        0.3550 11667780    0.78%           rho_7 U 2381660 1252936     30
 2394361 244112        cutoff              0.3578        0.3550 1

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:39 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3621.
Tried aggregator 1 time.
Reduced MIP has 40 rows, 42 columns, and 117 nonzeros.
Reduced MIP has 19 binaries, 21 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 563660 183305        0.3536    10        0.3585        0.3478  2619373    3.00%           rho_9 U 563660 563658     37
 578666 189042        0.3534    11        0.3585        0.3478  2682594    2.99%          rho_10 D 578666 578664     36
 593095 194346        cutoff              0.3585        0.3478  2749352    2.99%         alpha_8 U 593095 593093     30
 606639 197672        0.3539     9        0.3585        0.3479  2809822    2.97%           rho_1 D 606639 606637     33
 620747 201252        0.3551     2        0.3585        0.3479  2871776    2.96%          rho_12 U 620747 544109     26
 634675 204154        0.3497    10        0.3585        0.3480  2931664    2.94%          rho_10 D 634675 634674     29
Elapsed time = 174.81 sec. (168114.43 ticks, tree = 59.90 MB, solutions = 4)
 648809 208752        0.3489    13        0.3585        0.3480  2994167    2.93%           rho_7 D 648809 407809     34
 662601 212332        cutoff              0.3585        0.3481  3058236    2.91%   

 2148599 285532        cutoff              0.3585        0.3535  9485974    1.41%           rho_2 U 2148599 2148597     41
Elapsed time = 514.81 sec. (625901.37 ticks, tree = 89.12 MB, solutions = 4)
 2160904 282444        cutoff              0.3585        0.3535  9540159    1.39%           rho_3 D 2160904 441185     37
 2172940 280388        0.3570     6        0.3585        0.3536  9587120    1.38%           rho_4 U 2172940 2172939     42
 2185097 277552        0.3552     8        0.3585        0.3536  9634469    1.37%           rho_2 D 2185097 2185096     32
 2196558 275765        0.3542    11        0.3585        0.3537  9679802    1.35%           rho_3 D 2196558 2196556     26
 2208213 272653        0.3571    11        0.3585        0.3537  9725944    1.34%           rho_4 U 2208213 2208212     28
 2220086 271123        cutoff              0.3585        0.3538  9774929    1.32%           rho_8 D 2220086 2220085     50
 2231617 268508        cutoff              0.3585        0.3538

In [16]:
riskslim_summary['FL_score'], np.mean(riskslim_summary['KY_validation_score'])

(0.6370804703197479, 0.6637839158426052)

In [27]:
#### save results
summary_drug2_ky_inter_model = {"CART": cart_summary,
                                "EBM": ebm_summary,
                                "Stumps": stumps_summary,
                                "RiskSLIM": riskslim_summary}

In [28]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Two Year\\"
results = [["Drug",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_score'], 3)]]
with open(path + 'Two Year KY_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)